# Decision Trees

Decision tree is a tree like structure followed by the algorithm to reach at some conclusion node, starting from the root node.
The figure below shows a pictoral representation of a decision tree:
<img src='https://cdn-images-1.medium.com/max/690/1*xzF10JmR3K0rnZ8jtIHI_g.png'>


As shown in the figure above, lets say an offer comes up with $80,000 salary, a commute of 30 minutes, but free coffee. Then the algorithm goess as follows:
* Decision at the root node : yes
* Decision at the branch node: yes
* Decision at the second branch node: no
* Final Result at the leaf node: decline offer 

# Entropy

How does the algorithm know what valuees to put at what nodes and which feature to split on? Thats decided on the information gain at each node, calculated using entropy

<img src='http://dni-institute.in/blogs/wp-content/uploads/2015/08/Entrop.png'>

<img src='https://image.slidesharecdn.com/decisiontree-160904075506/95/decision-tree-17-638.jpg?cb=1472975726'>

Entropy can be defined as the 'randomness' of the split. For a particular split, the more the entropy, the lesser clarity we have about the seperation of data.

When we subtract the weighted average entropy of the child nodes from the parent entropy, we get a value which is known as information gain.

The decision tree being a greedy algorithm, it will iterate over every feature and value and calculate the information gain. The value and feature with the most information gain will be selected and the tree will grow further.


### Advantages:
- Its damn fast once built. (O(depth) running time)
- It gives fairly accurate results for less number of instances
- Its easy to understand for the user and has a pretty good pictorial representation

### Disadvantages
- The tree is mostly always prone to over=fitting i.e. it literally rotes the training data and does not generalize over new data. Hence, the low accruacy compared to other models.

### Optimizations
- **Pruning** - Pruning means cutting down some branches, where we feel that the tree is overfitting the  data. We are basically controlling the depth(or height?) of the tree by adding an extra depth parameter.
- **Random Forests** - Random Forests is an ensemble method for reducing overfitting. A random forest is a group of decision trees tested on random subsets of the training set and random subsets of the features set. The results of all the trees are taken and a majority vote(classification) or average(regression) is taken for the final result.

For more information on decision trees and the basic algorithm, surely have a look [here](https://www.youtube.com/watch?v=eKD5gxPPeY0)

### Our training Set

Our training set comes from the very first video game releassed by Pokemon: Pokemon Red. 
Each of the datapoint has 2 features and a result. The first feature is a string which describes the type of the starter pokemon of the player. The second feature is an int which describes the level of the starter pokemon. The result is a 1/0 where 1 means victory in Brock's Gym and 0 means a loss

<img src='http://oyster.ignimgs.com/mediawiki/apis.ign.com/pokemon-blue-version/9/90/Pokem_33.jpg'>
#### Salute to the classics

In [105]:
#Example dataset
data = [['Fire',12,0],['Fire',28,1],['Fire',15,0],['Fire',5,0],['Fire',20,1],
        ['Water',5,0],['Water',12,1],['Water',20,1],['Water',10,1],
        ['Grass',6,0],['Grass',10,0],['Grass',12,1],['Grass',15,1]]

A Node can be either a leaf node(result node) or a branch node(decision node) or the root node(original node).
A node contains the following:
- A result(1/0) if its a leaf node
- A best feature which is a feature to divide the tree further
- A best value which is the value of the feature on which to divide the tree
    - It is either a number which then divides on greater/less than basis
    - A list of classes if it is a string. This is for more than 2 branches from a node 
- Children - Its the list of further branches

In [106]:
#defining a class
class Node:
    def __init__(self,best_feature=-1, best_value=None, result=None, children=None):
        self.best_feature = best_feature
        self.best_value = best_value
        self.result = result
        self.children = children

In [107]:
from math import log

In [108]:
#Calculate entropy function
def Entropy(rows):
    p=0
    if len(rows) != 0:  #No need to calculate entropy for an empty list
        for i in rows:
            p = p + i[-1] 
        p = p / len(rows)

    if(p==0 or p==1): # Pure subset, entropy =0
        return 0
    else:
        entropy = -1*(p)*(log(p)/log(2)) - (log(1-p)/log(2))*(1-p)  # Entropy function
        return entropy

In [109]:
#Split the rows in sub branches
def split(rows, col, value=None):
    spl=[]
    
    if value==None:  #discrete data
        col_values = set([row[col] for row in rows])
        for i in col_values:
            spl.append( [row for row in rows if row[col]==i] )  # Split of the data created wrt the feature values
    
    else: #Continuous data
        spl.append( [row for row in rows if row[col] >= value] )
        spl.append( [row for row in rows if row[col] < value] )  # Split on basis of greater/lesser than value
        
    return spl

In [110]:
#Returns a list of unique labels of a column
def unique(rows,col):
    res=[]
    for row in rows:
        if row[col] not in res:
            res.append(row[col])
    return res

In [111]:
#The main builder function
def build_tree(rows):
    if len(rows) <=1:
        return Node(best_feature = rows[0][0], result = rows[0][-1])
    
    node_entropy = Entropy(rows) #Calculate the node entropy
    
    if node_entropy == 0:  #Perfect subset- Make a leaf node
        #print(rows[0][-1])
        return Node(result = rows[0][-1])
    
    #print('node entropy:  ',node_entropy,'\n')
    gain=0
    best_feature = None
    best_value = None
    bestchildren=None
    
    for col in range(len(rows[0]) - 1):  #Iterate for every column except result col
        #print('col: ', col, '\n')
        col_values = unique(rows,col)  #Unique labels of each col
        #print('col value:', col_values,'\n')
        if isinstance(col_values[0], str):  #if string then discrete and division likewise
            sub_branches = split(rows,col)  #split into children branches to test entropy gain
            #print('sub branches: ',sub_branches,'\n')
            entropy=0
            #print('entropy before: ', entropy, '\n')
            for i in range(len(sub_branches)):
                # total child entropy is the weighted average of each of the sub-branches' entropy
                entropy = entropy + (len(sub_branches[i])/len(rows))*Entropy(sub_branches[i]) #Total entropy of the children
            #print('entropy after: ',entropy , '\n')
            #the information gain for this child
            ch_gain = node_entropy - entropy
            #print('ch_gain, gain', ch_gain,gain, '\n')
                
            #Updating best feature, best value, and best sub-branches
            if ch_gain>gain:
                gain = ch_gain
                best_feature= col
                best_value = col_values
                bestchildren = sub_branches
        
        else:  #Continuous data, split on  a value
            for value in col_values:
                #print('Value: ', value, '\n')
                sub_branches = split(rows,col,value)  #split in 2 children, greater or less than value
                #print('Subbranches: ',sub_branches,'\n')
                entropy = (len(sub_branches[0])/len(rows)) * Entropy(sub_branches[0]) + (len(sub_branches[1])/len(rows))*Entropy(sub_branches[1])
                #print('entropy: ',entropy,'\n')
                ch_gain = node_entropy - entropy  #the gain of this particular child
                #print('ch_gain, gain : ',ch_gain,gain,'\n')
                #Updating best feature and best value
                if ch_gain>gain:
                    gain = ch_gain
                    best_feature = col
                    best_value = value
                    bestchildren = sub_branches
                
    if gain == node_entropy:  #100% pure data
        #print(bestchildren)
        return Node(best_feature = best_feature, best_value = best_value, children=[build_tree(bst) for bst in bestchildren])
    else:
        #print(bestchildren)
        children=[build_tree(bst) for bst in bestchildren]
        return Node(best_feature = best_feature, best_value = best_value, children = children)


In [112]:
#Returns the result for a testing datapoint X
def predict(model, X):
    if model.result != None:  #a leaf node
        return model.result
    
    elif isinstance(model.best_value,int) or isinstance(model.best_value,float):  #if best_feature is numeric
        
        if X[model.best_feature] >= model.best_value: #Select the bigger branch
            return predict(model.children[0], X)  
        
        else:  #Select the lower branch
            return predict(model.children[1], X)
        
    else:  #The best feature has discrete values
        idx = (model.best_value).index(X[model.best_feature])  #index of the particular label
        return predict((model.children)[idx],X)  #Go to the label branch         
        
        

In [113]:
#Building the model
model = build_tree(data)

In [117]:
#Predicting 
predict(model,['Fire',21])

1

**NOTE** : This tree will work for any data, but for more data, it will overfit, so convert it into a random forest by making k such trees which work on m random examples using n features .